<a href="https://colab.research.google.com/github/parveen180/ML-/blob/main/Deploying_ML_model_as_Public_API_using_FastAPI_and_Ngrok_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastapi

In [2]:
!pip install uvicorn

In [3]:
!pip install pickle5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pickle5
  Running setup.py clean for pickle5
Failed to build pickle5
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pickle5)


In [4]:
!pip install pydantic

In [5]:
!pip install requests

In [6]:
!pip install pyngrok

In [7]:
!pip install nest asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.4/241.4 kB 8.4 MB/s eta 0:00:00


In [8]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import pickle
import numpy as np
import pandas as pd
from pydantic import BaseModel
import requests
import nest_asyncio
from pyngrok import ngrok
import json

In [9]:
app=FastAPI()

In [10]:
origins=["*"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


In [11]:
class model_input(BaseModel):
    Pregnancies	: int
    Glucose	: int
    BloodPressure : int
    SkinThickness:int
    Insulin:int
    BMI:float
    DiabetesPedigreeFunction:float
    Age : int

In [12]:
model=pickle.load(open('/content/Trained_model.sav','rb'))

In [13]:
@app.post('/diabetes_prediction')
def diabetes_predd(input_parameters : model_input):
  input_data=input_parameters.json()
  input_dictionary=json.loads(input_data)

  preg=input_dictionary['Pregnancies']
  glu=input_dictionary['Glucose']
  bp=input_dictionary['BloodPressure']
  skin=input_dictionary['SkinThickness']
  ins=input_dictionary['Insulin']
  bmi=input_dictionary['BMI']
  dpf=input_dictionary['DiabetesPedigreeFunction']
  age=input_dictionary['Age']

  input_list=[preg,glu,bp,skin,ins,bmi,dpf,age]

  prediction=model.predict([input_list])

  if (prediction[0]==0):
    return 'The person is not diabetic'
  else:
    return 'The person is diabetic'


In [14]:
from pyngrok import ngrok

In [15]:
ngrok.set_auth_token("333ANIy0Iack1u99AHhZPNUivpy_5GN5f2ZykbsQ61KNwRZBB")
ngrok_tunnel=ngrok.connect(8000)
print('Public URL :' , ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app,port=8000)

Public URL : https://4f42e8a945cd.ngrok-free.app


INFO:     Started server process [523]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     103.175.137.196:0 - "POST /diabetes_prediction HTTP/1.1" 200 OK


/tmp/ipython-input-1320221246.py:3: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  input_data=input_parameters.json()
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [523]
